In [1]:
# import os

# os.environ["CUDA_VISIBLE_DEVICES"]= "2,3"

In [2]:
import torch
model = torch.hub.load("pytorch/vision", "vit_b_16")

import torchvision
from Models.transformer import VisionTransformer as vit
import Models.Conv as conv

from DataLoader import CIFAR100
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn

import utils

import numpy as np
import random

/opt/conda/envs/cuda_11_6/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using cache found in /root/.cache/torch/hub/pytorch_vision_main
/root/.cache/torch/hub/pytorch_vision_main/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")
Using cache found in /root/.cache/torch/hub/pytorch_vision_main
/root/.cache/torch/hub/pytorch_vision_main/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
train_loader, test_loader = CIFAR100.get_data(512, use_original=True)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
def get_network(name, device):
    """ return given network
    """

    if name == 'vgg16':
        from models.vgg import vgg16_bn
        net = vgg16_bn()
    elif name == 'vgg13':
        from models.vgg import vgg13_bn
        net = vgg13_bn()
    elif name == 'vgg11':
        from models.vgg import vgg11_bn
        net = vgg11_bn()
    elif name == 'vgg19':
        from models.vgg import vgg19_bn
        net = vgg19_bn()
    elif name == 'densenet121':
        from models.densenet import densenet121
        net = densenet121()
    elif name == 'densenet161':
        from models.densenet import densenet161
        net = densenet161()
    elif name == 'densenet169':
        from models.densenet import densenet169
        net = densenet169()
    elif name == 'densenet201':
        from models.densenet import densenet201
        net = densenet201()
    elif name == 'googlenet':
        from models.googlenet import googlenet
        net = googlenet()
    elif name == 'inceptionv3':
        from models.inceptionv3 import inceptionv3
        net = inceptionv3()
    elif name == 'inceptionv4':
        from models.inceptionv4 import inceptionv4
        net = inceptionv4()
    elif name == 'inceptionresnetv2':
        from models.inceptionv4 import inception_resnet_v2
        net = inception_resnet_v2()
    elif name == 'xception':
        from models.xception import xception
        net = xception()
    elif name == 'resnet18':
        from models.resnet import resnet18
        net = resnet18()
    elif name == 'resnet34':
        from models.resnet import resnet34
        net = resnet34()
    elif name == 'resnet50':
        from models.resnet import resnet50
        net = resnet50()
    elif name == 'resnet101':
        from models.resnet import resnet101
        net = resnet101()
    elif name == 'resnet152':
        from models.resnet import resnet152
        net = resnet152()
    elif name == 'preactresnet18':
        from models.preactresnet import preactresnet18
        net = preactresnet18()
    elif name == 'preactresnet34':
        from models.preactresnet import preactresnet34
        net = preactresnet34()
    elif name == 'preactresnet50':
        from models.preactresnet import preactresnet50
        net = preactresnet50()
    elif name == 'preactresnet101':
        from models.preactresnet import preactresnet101
        net = preactresnet101()
    elif name == 'preactresnet152':
        from models.preactresnet import preactresnet152
        net = preactresnet152()
    elif name == 'resnext50':
        from models.resnext import resnext50
        net = resnext50()
    elif name == 'resnext101':
        from models.resnext import resnext101
        net = resnext101()
    elif name == 'resnext152':
        from models.resnext import resnext152
        net = resnext152()
    elif name == 'shufflenet':
        from models.shufflenet import shufflenet
        net = shufflenet()
    elif name == 'shufflenetv2':
        from models.shufflenetv2 import shufflenetv2
        net = shufflenetv2()
    elif name == 'squeezenet':
        from models.squeezenet import squeezenet
        net = squeezenet()
    elif name == 'mobilenet':
        from models.mobilenet import mobilenet
        net = mobilenet()
    elif name == 'mobilenetv2':
        from models.mobilenetv2 import mobilenetv2
        net = mobilenetv2()
    elif name == 'nasnet':
        from models.nasnet import nasnet
        net = nasnet()
    elif name == 'attention56':
        from models.attention import attention56
        net = attention56()
    elif name == 'attention92':
        from models.attention import attention92
        net = attention92()
    elif name == 'seresnet18':
        from models.senet import seresnet18
        net = seresnet18()
    elif name == 'seresnet34':
        from models.senet import seresnet34
        net = seresnet34()
    elif name == 'seresnet50':
        from models.senet import seresnet50
        net = seresnet50()
    elif name == 'seresnet101':
        from models.senet import seresnet101
        net = seresnet101()
    elif name == 'seresnet152':
        from models.senet import seresnet152
        net = seresnet152()
    elif name == 'wideresnet':
        from models.wideresidual import wideresnet
        net = wideresnet()
    elif name == 'stochasticdepth18':
        from models.stochasticdepth import stochastic_depth_resnet18
        net = stochastic_depth_resnet18()
    elif name == 'stochasticdepth34':
        from models.stochasticdepth import stochastic_depth_resnet34
        net = stochastic_depth_resnet34()
    elif name == 'stochasticdepth50':
        from models.stochasticdepth import stochastic_depth_resnet50
        net = stochastic_depth_resnet50()
    elif name == 'stochasticdepth101':
        from models.stochasticdepth import stochastic_depth_resnet101
        net = stochastic_depth_resnet101()

    else:
        print('the network name you have entered is not supported yet')
        sys.exit()

    net = net.to(device)

    return net

In [7]:
import models
teacher = models.__dict__['resnet110'](num_classes=100)
teacher.load_state_dict(torch.load("saved_models/small_model/resnet110/best.pth").state_dict())

# torch.load("saved_models/resnet20/best.pth")
# import models
# # teacher = models.__dict__['resnet20'](num_classes=100)
# teacher = torch.load("saved_models/resnet20/best.pth")

<All keys matched successfully>

In [8]:
device = "cuda"

teacher = teacher.to(device)
# teacher = torch.nn.DataParallel(teacher, device_ids=[0, 1])

In [9]:
utils.test(teacher, test_loader,device)


0 	 test acc : 0.7408999800682068


tensor(0.7409, device='cuda:0')

In [10]:
criterion_response = lambda a,b : criterion_KLD(torch.log_softmax(a, dim=1),torch.softmax(b, dim=1))
criterion_KLD = torch.nn.KLDivLoss(reduction="batchmean")
criterion_L1 = lambda a,b: torch.sum(a-b)/len(a)
criterion_onlylabel = lambda a,b : mse(a*b, b)
criterion_CE = nn.CrossEntropyLoss()
criterion_MSE = nn.MSELoss()


softmax = torch.nn.Softmax(dim = 1)

In [11]:
T_optimizer = optim.SGD(teacher.parameters(), lr=0.01, momentum=0.9)

In [16]:
best_acc = 0.0
stack = 0
accs_train = []
accs_test = []

for mean in [0.5, 0.2, 0.3, 0.4, 0.5,0.6,0.7,0.8,0.9,1.0,1.1, 1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9]:
    for std in [0.01, 0.003, 0.006, 0.008, 0.01, 0.05, 0.1, 0.2]:
        utils.set_seed()
        T_correct = 0
        all_data = 0
        utils.set_seed()
        print(f"mean : {mean} \t std: {std}")

        pbar = tqdm(train_loader)
        for i, (img, label) in enumerate(pbar):
            teacher.train()
            input_data = img.to(device)
            label = label.to(device)


            all_data += len(input_data)
            input_lrp = utils.get_LRP_img(input_data, label, teacher, criterion_CE, T_optimizer, mean=0.5, std = 0.01, mult = 0.4).cuda()
            T_optimizer.zero_grad()

    #         layer = random.randint(0,  2+encoder_length)

            output_T = teacher(input_lrp)
            if isinstance(output_T, tuple):
                output_T = output_T[0]

            T_correct += sum(label == torch.argmax(output_T, dim=1))
            
    #         layer = random.randint(0,  2+encoder_length)
            pbar.set_postfix({"TeacaherACC" : (T_correct / all_data).item()})
        print(T_correct / all_data)
        accs_train.append((T_correct / all_data).item())

mean : 0.5 	 std: 0.01


100%|██████████| 98/98 [01:06<00:00,  1.48it/s, TeacaherACC=0.952]


tensor(0.9525, device='cuda:0')
mean : 0.5 	 std: 0.003


100%|██████████| 98/98 [01:05<00:00,  1.49it/s, TeacaherACC=0.952]


tensor(0.9525, device='cuda:0')
mean : 0.5 	 std: 0.006


...

tensor(0.9525, device='cuda:0')
mean : 1.9 	 std: 0.1


100%|██████████| 98/98 [01:03<00:00,  1.53it/s, TeacaherACC=0.952]


tensor(0.9525, device='cuda:0')
mean : 1.9 	 std: 0.2


100%|██████████| 98/98 [01:06<00:00,  1.48it/s, TeacaherACC=0.952]

tensor(0.9525, device='cuda:0')


In [11]:
# train data 
# acc : tensor(0.9537, device='cuda:0')
# mean : 0.5 	 std: 0.01

# test도 mean이 0.5 std가 0.01일때가 최대인듯



In [3]:
best_acc = 0.0
stack = 0
accs_train = []
accs_test = []

for mult in [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008]:
    T_correct = 0
    all_data = 0

    print(f"mult : {mult}")
    with tqdm(test_loader, unit='batch') as pbar:
        for i, (img, label) in enumerate(test_loader):
            teacher.eval()
            input_data = img.to(device)
            label = label.to(device)


            all_data += len(input_data)
            input_lrp = utils.get_LRP_img_plus(input_data, label, teacher, criterion_CE, T_optimizer, mult).cuda()

            T_optimizer.zero_grad()

    #         layer = random.randint(0,  2+encoder_length)

            output_T = teacher(input_lrp)
            if isinstance(output_T, tuple):
                output_T = output_T[0]

            T_correct += sum(label == torch.argmax(output_T, dim=1))
            pbar.set_postfix(TeacaherACC = T_correct / all_data)
        print(T_correct / all_data)

mult : 0.001


NameError: name 'test_loader' is not defined